# Image Dataset Creator for the project Contactless Office Entry And Attendance
#### @Authors - Sumit Kumar Rai & Shreyas Lall

Getting images from the employees to train the model is one of the hectic tasks. So with this file we are trying to automate this process.
This files is specially meant for google colab as it can be run on chrome browser and does not require someone to download all the necessary packages.
The steps to do so are as follows:

1. Mount the google drive
2. Sit infront of the webcam and run the next cell.
3. Wait for the cell to terminate on it's own
4. Enter your first name when asked
5. The file will be downloaded to your google drive. Now you can easily share it


In [1]:
#Mount the google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
#import the libraries

from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2 as cv
import numpy as np
import PIL
import io
import html
import time
import os

## Helper Functions

#function to convert the Javascript code into an OpenCV image

def js_to_image(js_reply):
  """
  Params:
        js_reply: Javascript object containing image from webcam

  Returns:
        img: OpenCV BGR image

  """
  #decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  #convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes,dtype=np.uint8)
  #decode numpy array into OpenCV BGR image
  img = cv.imdecode(jpg_as_np,flags=1)

  return img


# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

name = str(input('ENTER YOUR FIRST NAME: '))
#location = str(input('Enter the location you want to save\n\n Steps to do so :\n 1. Mount the drive \n'))
location = '/gdrive/MyDrive/'+name

try:
  os.mkdir(location)

except:
  print('Directory Already Exist')
  print('Overwriting....')


video_stream()
os.chdir(location)
print(os.listdir(location))
label_html = 'Say Cheese you are being captured...'
bbox = ''
img_number = 0
max_images = 10
while True:
  if img_number >= max_images:
    print('Breaking the loop')
    break

  else:
      js_reply = video_frame(label_html,bbox)
      if not js_reply:
        print('Nothing Found')
        break

      img = js_to_image(js_reply['img'])
      filename = 'sumit_'+str(img_number)+'.jpg'
      cv.imwrite(filename,img)
      print('Success')
      img_number +=1
      if cv.waitKey(0) & 0xFF == 27:
        break






ENTER YOUR FIRST NAME: bvb


<IPython.core.display.Javascript object>

[]
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Breaking the loop


In [5]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [20]:
name = str(input('ENTER YOUR FIRST NAME: '))
#location = str(input('Enter the location you want to save\n\n Steps to do so :\n 1. Mount the drive \n'))
location = '/content/drive/MyDrive/'+name

try:
  os.mkdir(location)

except:
  print('Directory Already Exist')


video_stream()
os.chdir(location)
print(os.listdir(location))
label_html = 'Say Cheese you are being captured...'
bbox = ''
img_number = 0
max_images = 10
while True:
  if img_number >= max_images:
    print('Breaking the loop')
    break

  else:
      js_reply = video_frame(label_html,bbox)
      if not js_reply:
        print('Nothing Found')
        break

      img = js_to_image(js_reply['img'])
      filename = 'sumit_'+str(img_number)+'.jpg'
      cv.imwrite(filename,img)
      print('Success')
      img_number +=1
      if cv.waitKey(0) & 0xFF == 27:
        break


ENTER YOUR FIRST NAME: sumit
File Already Exist


<IPython.core.display.Javascript object>

[]
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Breaking the loop
